In [34]:
import quandl
quandl.ApiConfig.api_key = "KxKMd8dsHx1sWnaGj6_U"
import numpy as np
import matplotlib as plt
import pandas as pd
import keras
from sklearn.preprocessing import MinMaxScaler

In [35]:
def get_monthly_close_prices(start_date,end_date):
    
    df = quandl.get_table('SHARADAR/SEP',date={'gte':start_date,'lte':end_date}, qopts={'columns':['ticker','date','close']},paginate=True).fillna(value=0)
    close = df.pivot(index='date', columns='ticker', values='close')
    monthly = close.resample('BM', how=lambda x: x[-1])
    #.agg(lambda x: x[-1])
    return monthly

In [36]:
monthly_prices = get_monthly_close_prices('2018-06-01','2019-01-01')

/usr/bin/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:5: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...)..apply(<func>)
  """


In [37]:
def get_monthly_returns(monthly_prices):
    """
    Compute returns for each ticker and date in close.
    
    Parameters
    ----------
    close : DataFrame
        Close prices for each ticker and date
    
    Returns
    -------
    returns : DataFrame
        Returns for each ticker and date
    """
        

    #.agg(lambda x: x[-1])
    monthly_returns = (monthly_prices - monthly_prices.shift(1))/monthly_prices.shift(1).fillna(value=0)
    
    
    return monthly_returns

In [38]:
monthly_retuns_june_dec_18 = get_monthly_returns(monthly_prices)


In [39]:
monthly_retuns_june_dec_18.head()

ticker,A,AA,AAC,AAGIY,AAL,AALBF,AAMC,AAME,AAN,AAOI,...,ZSAN,ZTO,ZTS,ZUMZ,ZUO,ZURVY,ZX,ZYME,ZYNE,ZYXI
date,,,,,,,,,,,,,,,,,,,,,
2018-06-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-07-31,0.067917,-0.077005,0.108858,0.003711,0.041623,NaN,-0.060215,-0.037037,-0.003222,-0.144098,...,0.024510,-0.008500,0.015143,-0.095808,-0.098162,0.039634,NaN,-0.161705,-0.317248,-0.035484
2018-08-31,0.022714,0.032355,-0.147257,-0.019909,0.023773,NaN,-0.053394,0.057692,0.148003,0.076243,...,-0.002392,-0.069087,0.047641,0.375276,0.124338,-0.008472,NaN,0.133979,0.207519,0.030100
2018-09-28,0.044418,-0.095590,-0.138826,0.033082,0.020998,-0.055386,-0.035455,-0.090909,0.095334,-0.403772,...,-0.019185,-0.102384,0.010596,-0.154093,-0.162074,0.034834,NaN,0.117438,0.016189,-0.032468
2018-10-31,-0.081514,-0.133911,-0.277851,-0.152528,-0.151222,0.000000,-0.181287,0.120000,-0.134594,-0.203974,...,-0.171149,-0.021123,-0.015400,-0.117268,-0.116400,-0.016196,NaN,-0.186624,-0.355392,0.030201


In [40]:
monthly_retuns_june_dec_18 = monthly_retuns_june_dec_18.iloc[ 1: , : ]

In [41]:
monthly_retuns_june_dec_18 = monthly_retuns_june_dec_18.fillna(0)

In [42]:
training_returns = monthly_retuns_june_dec_18.iloc[ :5 , : ]

In [43]:
target_returns = monthly_retuns_june_dec_18.iloc[ 5:6 , : ]

In [44]:
def get_factor_data(start_date,end_date):
    
    df = quandl.get_table('SHARADAR/SF1',dimension='ARQ',calendardate={'gte':start_date, 'lte':end_date}, paginate=True).fillna(value=0)

    
    return df

In [45]:
#quandl.get_table('SHARADAR/SF1',dimension='ARY',calendardate={'gte':'2018-06-01', 'lte':'2019-01-01'}, paginate=True)

In [46]:
factors_june_dec_18 = get_factor_data('2018-06-01','2019-01-01')

In [47]:
factors_june_dec_18.head

<bound method NDFrame.head of       ticker dimension calendardate    datekey reportperiod lastupdated  \
None                                                                      
0       ZYXI       ARQ   2018-12-31 2019-02-26   2018-12-31  2019-02-26   
1       ZYXI       ARQ   2018-09-30 2018-11-06   2018-09-30  2019-02-26   
2       ZYXI       ARQ   2018-06-30 2018-08-02   2018-06-30  2019-02-26   
3       ZYNE       ARQ   2018-12-31 2019-03-11   2018-12-31  2019-03-11   
4       ZYNE       ARQ   2018-09-30 2018-11-08   2018-09-30  2019-03-11   
5       ZYNE       ARQ   2018-06-30 2018-08-02   2018-06-30  2019-03-11   
6       ZYME       ARQ   2018-12-31 2019-03-06   2018-12-31  2019-03-06   
7       ZYME       ARQ   2018-09-30 2018-11-06   2018-09-30  2019-03-06   
8       ZYME       ARQ   2018-06-30 2018-08-01   2018-06-30  2019-03-06   
9        ZUO       ARQ   2018-09-30 2018-12-13   2018-10-31  2018-12-13   
10       ZUO       ARQ   2018-06-30 2018-09-12   2018-07-31  2018-12-1

In [48]:
monthly_retuns_june_dec_18.head() #['date'].drop_duplicates().sort_values()

ticker,A,AA,AAC,AAGIY,AAL,AALBF,AAMC,AAME,AAN,AAOI,...,ZSAN,ZTO,ZTS,ZUMZ,ZUO,ZURVY,ZX,ZYME,ZYNE,ZYXI
date,,,,,,,,,,,,,,,,,,,,,
2018-07-31,0.067917,-0.077005,0.108858,0.003711,0.041623,0.000000,-0.060215,-0.037037,-0.003222,-0.144098,...,0.024510,-0.008500,0.015143,-0.095808,-0.098162,0.039634,0.0,-0.161705,-0.317248,-0.035484
2018-08-31,0.022714,0.032355,-0.147257,-0.019909,0.023773,0.000000,-0.053394,0.057692,0.148003,0.076243,...,-0.002392,-0.069087,0.047641,0.375276,0.124338,-0.008472,0.0,0.133979,0.207519,0.030100
2018-09-28,0.044418,-0.095590,-0.138826,0.033082,0.020998,-0.055386,-0.035455,-0.090909,0.095334,-0.403772,...,-0.019185,-0.102384,0.010596,-0.154093,-0.162074,0.034834,0.0,0.117438,0.016189,-0.032468
2018-10-31,-0.081514,-0.133911,-0.277851,-0.152528,-0.151222,0.000000,-0.181287,0.120000,-0.134594,-0.203974,...,-0.171149,-0.021123,-0.015400,-0.117268,-0.116400,-0.016196,0.0,-0.186624,-0.355392,0.030201
2018-11-30,0.116685,-0.090883,-0.642468,0.081869,0.144812,0.000000,-0.012041,0.010714,-0.007002,0.048905,...,0.103245,0.057953,0.041265,-0.152193,-0.068071,0.011620,0.0,0.007831,-0.083650,0.009772


In [49]:
monthly_retuns_june_dec_18.pivot(index='ticker', columns='ticker', values='close')

KeyError: 'ticker'

In [ ]:
factors_june_dec_18['calendardate'].drop_duplicates().sort_values()

In [ ]:
factors_june_dec_18.shape

In [ ]:
import keras.losses
import tensorflow as tf

# Create loss function
def sign_penalty(y_true, y_pred):
    penalty = 100.
    loss = tf.where(tf.less(y_true * y_pred, 0), \
                     penalty * tf.square(y_true - y_pred), \
                     tf.square(y_true - y_pred))

    return tf.reduce_mean(loss, axis=-1)

keras.losses.sign_penalty = sign_penalty  # enable use of loss with keras
print(keras.losses.sign_penalty)

In [ ]:
corr_matrix = df.corr().abs()

In [ ]:
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper

In [ ]:
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [ ]:
factors_june_dec_18_reduced = factors_june_dec_18.drop(factors_june_dec_18[to_drop], axis=1)

In [ ]:
factors_june_dec_18_reduced[factors_june_dec_18_reduced['calendardate'] != '2018-12-31'] 

In [ ]:
factors_june_dec_18[factors_june_dec_18['calendardate'] != '2018-12-31']

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
factors_june_dec_18['calendardate'] != '2018-12-31'

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
factors_june_dec_18_reduced['calendardate']

In [ ]:



features = factors_june_dec_18_reduced
target = monthly_retuns_june_dec_18

In [ ]:
features.rename(columns={"calendardate": "date"})

In [ ]:
from sklearn.preprocessing import scale

# Remove unimportant features (weekdays)
train_features = train_features.iloc[:, :-4]
test_features = test_features.iloc[:, :-4]

# Standardize the train and test features
scaled_train_features = scale(train_features)
scaled_test_features = scale(test_features)

# Plot histograms of the 14-day SMA RSI before and after scaling
f, ax = plt.subplots(nrows=2, ncols=1)
train_features.iloc[:, 2].hist(ax=ax[0])
ax[1].hist(scaled_train_features[:, 2])
plt.show()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

# Create the model
model_1 = Sequential()
model_1.add(Dense(100, input_dim=scaled_train_features.shape[1], activation='relu'))
model_1.add(Dense(20, activation='relu'))
model_1.add(Dense(1, activation='linear'))

# Fit the model
model_1.compile(optimizer='adam', loss='mse')
history = model_1.fit(scaled_train_features, train_targets, epochs=25)